In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!pip install transformers datasets accelerate peft trl bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beha

In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from trl import SFTConfig, SFTTrainer

# Model and dataset setup
model_name = "meta-llama/Llama-2-7b-chat-hf"  # LLaMA 2 7B
dataset_name = "your_custom_dataset_name"     # Replace with your dataset name
dataset_text_field = "text"                   # The field in your dataset containing text

# Load custom dataset

In [ ]:
dataset = load_dataset("text", data_files={"train": ["pdf1.txt", "pdf2.txt"]})

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:


# 4-bit Quantization Configuration with CPU Offloading
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                      # Use 4-bit quantization
    bnb_4bit_use_double_quant=True,         # Use nested quantization for better memory usage
    bnb_4bit_quant_type="nf4",              # NormalFloat4 quantization type
    bnb_4bit_compute_dtype=torch.float16,  # Use float16 for computation
    load_in_8bit_fp32_cpu_offload=True,     # Enable CPU offloading
)

# Load LLaMA 2 tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load LLaMA 2 model with 4-bit quantization and CPU offloading
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,         # Apply quantization config
    device_map="auto",                      # Distribute across available devices
    trust_remote_code=True,
    offload_buffers=True                    # Enable offloading of buffers
)

# Prepare the model for LoRA fine-tuning
lora_config = LoraConfig(
    r=8,                                    # Rank of the LoRA update matrices
    lora_alpha=16,                          # Alpha parameter
    lora_dropout=0.1,                       # Dropout probability
    task_type="CAUSAL_LM"                   # Task is next-token prediction (causal LM)
)

# Wrap the model with LoRA configuration
model = get_peft_model(model, lora_config)

Unused kwargs: ['load_in_8bit_fp32_cpu_offload']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:

# Set padding token as EOS token
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
# SFT configuration
sft_config = SFTConfig(
    dataset_text_field="text",
    max_seq_length=512,                     # Maximum sequence length
    output_dir="./llama_lora_finetune",     # Output directory for saving the model
    gradient_accumulation_steps=16,         # Accumulate gradients over multiple steps
    per_device_train_batch_size=4,          # Batch size per device
    learning_rate=2e-4,                     # Learning rate
    num_train_epochs=3,                     # Number of epochs
    logging_steps=10,                       # Logging steps
)

# Fine-tuning with SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    tokenizer=tokenizer,
    args=sft_config
)

# Start training
trainer.train()



Map:   0%|          | 0/4608 [00:00<?, ? examples/s]

Step,Training Loss
10,5.720500
20,4.092000
30,4.328500
40,3.189200
50,3.345200
60,2.659400
70,2.496000
80,2.703700
90,2.436000
100,2.642700


TrainOutput(global_step=216, training_loss=2.82858384097064, metrics={'train_runtime': 1638.4445, 'train_samples_per_second': 8.437, 'train_steps_per_second': 0.132, 'total_flos': 1.000473785745408e+16, 'train_loss': 2.82858384097064, 'epoch': 3.0})

In [ ]:
new_model="llama_finetune"
trainer.model.save_pretrained(new_model)

In [ ]:
# Empty VRAM
del trainer
import gc
gc.collect()
gc.collect()

24549

In [ ]:
from peft import LoraConfig, PeftModel
device_map = {"": 0}
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write)

In [ ]:


model.push_to_hub("vivivivivivi/Llama-2-7b-chat-finetune", check_pr=True)

tokenizer.push_to_hub("vivivivivivi/Llama-2-7b-chat-finetune",check_pr=True)

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/vivivivivivi/Llama-2-7b-chat-finetune/commit/f3d3940893a84be203e41eceba6f15b594d408c5', commit_message='Upload tokenizer', commit_description='', oid='f3d3940893a84be203e41eceba6f15b594d408c5', pr_url=None, pr_revision=None, pr_num=None)